In [13]:
# Import necessary libraries
import os
import cv2
import numpy as np
from ur3e_dataloader import UR3EDataset
from torch.utils.data import Dataset, DataLoader
import time


In [14]:
def visualize_batch(batch, batch_idx=0):
    """
    Visualizes a batch of data from the dataloader in a single window.

    Args:
        batch (dict): A batch of data from the dataloader.
        batch_idx (int): Index within the batch to visualize (default: 0).
    """
    # Extract batch data for the selected sample
    front_cam = batch["front_cam"][batch_idx].cpu().numpy()  # (3, H, W, 3)
    side_cam = batch["side_cam"][batch_idx].cpu().numpy()  # (3, H, W, 3)
    hand_cam = batch["hand_cam"][batch_idx].cpu().numpy()  # (3, H, W, 3)

    # Convert images to OpenCV format (H, W, C) and scale up for visibility
    def preprocess_img(img):
        """
        Preprocess image from [f, H, W, C] to OpenCV format [H, W, C] for display.
        """
        # img = np.transpose(img, (1, 2, 0))  # Convert [f, H, W, C] -> [H, W, f, C] -> [H, W, C]
        img = np.clip(img * 255, 0, 255).astype(np.uint8)  # Convert back to 0-255 for display
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                
        return img  # Return as H, W, C

    front_cam_imgs = [preprocess_img(front_cam[i]) for i in reversed(range(3))]
    side_cam_imgs = [preprocess_img(side_cam[i]) for i in reversed(range(3))]
    hand_cam_imgs = [preprocess_img(hand_cam[i]) for i in reversed(range(3))]

    top_row = np.concatenate(front_cam_imgs, axis=1)
    middle_row = np.concatenate(side_cam_imgs, axis=1)
    bottom_row = np.concatenate(hand_cam_imgs, axis=1)

    all_images = np.vstack([top_row, middle_row, bottom_row])

    # Get image dimensions
    H, W, _ = all_images.shape
    step_x = W // 3  # Width of each column
    step_y = H // 3  # Height of each row

    # Titles for rows
    row_titles = ["Front Cam", "Side Cam", "Hand Cam"]
    col_titles = ["t", "t-1", "t-2"]  # Reordered column labels

    font_scale = 0.6  # Reduce font size
    thickness = 1  # Reduce text thickness

    # Add row titles
    for i, title in enumerate(row_titles):
        y_pos = step_y * i + step_y // 2
        cv2.putText(all_images, title, (5, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 
                    font_scale, (0, 255, 0), thickness, cv2.LINE_AA)

    # Add column titles
    for i, title in enumerate(col_titles):
        x_pos = step_x * i + step_x // 2 - 15
        cv2.putText(all_images, title, (x_pos, 20), cv2.FONT_HERSHEY_SIMPLEX, 
                    font_scale, (0, 255, 0), thickness, cv2.LINE_AA)


    # Display the images in one window
    cv2.imshow("Camera Views: Front, Side, Hand (t-2, t-1, t)", all_images)
   

    # Optionally print numerical data for the current sample
    ee_pose = batch["ee_pose"][batch_idx].cpu().numpy()  # (3,7)
    ee_velocity = batch["ee_velocity"][batch_idx].cpu().numpy()  # (6)
    hole_pose = batch["hole_pose"][batch_idx].cpu().numpy()  # (7)
    state = batch["state"][batch_idx].cpu().numpy()  # (1)

    print("\nEnd Effector Pose Over Time:")
    print(f"  t-2: {ee_pose[0]}")
    print(f"  t-1: {ee_pose[1]}")
    print(f"  t:   {ee_pose[2]}")
    
    print("\nEnd Effector Velocity:")
    print(f"  [vx, vy, vz, wx, wy, wz]: {ee_velocity}")
    
    print("\nHole Position and Orientation:")
    print(f"  [x, y, z, qx, qy, qz, qw]: {hole_pose}")

    # Print the current state
    state_labels = {0: "Move", 1: "Insert", 2: "Rotate"}
    print(f"\nCurrent State: {state_labels.get(state.item(), 'Unknown')} ({state.item()})")

    cv2.waitKey(0)  # Wait for key press
    # cv2.destroyAllWindows()  # Close the window after key press


In [15]:
# Define the directory for the zip files (data storage directory)
zip_dir = "/home/tanakrit-ubuntu/ur3e_mujoco_tasks/scripts/data"

# Define the batch size
batch_size = 16

# Instantiate the dataset
dataset = UR3EDataset(zip_dir)
print(dataset.episode_dict)

# Create a dataloader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)


/home/tanakrit-ubuntu/ur3e_mujoco_tasks/scripts/data/metadata_20250407_134047_0.pkl
/home/tanakrit-ubuntu/ur3e_mujoco_tasks/scripts/data/metadata_20250407_134047_1.pkl
{'20250407_134047_0.zip': ['20250407_134047_0_0.pkl', '20250407_134047_0_1.pkl', '20250407_134047_0_2.pkl', '20250407_134047_0_3.pkl', '20250407_134047_0_4.pkl', '20250407_134047_0_5.pkl', '20250407_134047_0_6.pkl', '20250407_134047_0_7.pkl', '20250407_134047_0_8.pkl', '20250407_134047_0_9.pkl', '20250407_134047_0_10.pkl', '20250407_134047_0_11.pkl', '20250407_134047_0_12.pkl', '20250407_134047_0_13.pkl', '20250407_134047_0_14.pkl', '20250407_134047_0_15.pkl', '20250407_134047_0_16.pkl', '20250407_134047_0_17.pkl', '20250407_134047_0_18.pkl', '20250407_134047_0_19.pkl', '20250407_134047_0_20.pkl', '20250407_134047_0_21.pkl', '20250407_134047_0_22.pkl', '20250407_134047_0_23.pkl', '20250407_134047_0_24.pkl', '20250407_134047_0_25.pkl', '20250407_134047_0_26.pkl', '20250407_134047_0_27.pkl', '20250407_134047_0_28.pkl', '20

In [ ]:
for batch in dataloader:
    for i in range(batch_size):
        visualize_batch(batch, batch_idx=i) 
        time.sleep(0.2)

cv2.destroyAllWindows()

20250407_134047_0.zip 20250407_134047_0_0.pkl 20250407_134047_0_0.pkl20250407_134047_0.zip 20250407_134047_0.zip20250407_134047_0.zip20250407_134047_0_0.pkl  20250407_134047_0_32.pkl  
20250407_134047_0_48.pkl20250407_134047_0_30.pkl  20250407_134047_0_16.pkl20250407_134047_0_28.pkl
20250407_134047_0_46.pkl  20250407_134047_0_14.pkl20250407_134047_0_44.pkl
 20250407_134047_0_12.pkl
20250407_134047_0.zip 20250407_134047_0_33.pkl 20250407_134047_0_31.pkl20250407_134047_0.zip 20250407_134047_0_29.pkl
 20250407_134047_0.zip20250407_134047_0_1.pkl 20250407_134047_0_49.pkl 20250407_134047_0_47.pkl 20250407_134047_0_45.pkl20250407_134047_0.zip
  20250407_134047_0_0.pkl20250407_134047_0.zip 20250407_134047_0_17.pkl20250407_134047_0_34.pkl  20250407_134047_0_32.pkl 20250407_134047_0_30.pkl20250407_134047_0_15.pkl 
20250407_134047_0_13.pkl 20250407_134047_0_0.pkl
20250407_134047_0.zip 20250407_134047_0_50.pkl 20250407_134047_0_48.pkl 20250407_134047_0_46.pkl
20250407_134047_0.zip
 20250407_13404